# Installation und Import

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import plotly.express as px
import pandas as pd
import numpy as np

# Zusatzmaterial

In diesem Abgschnitt gibt es einige Python-Methoden, die in der Haupterklärung nicht ausführlich behandelt wurden. Diese Methoden können jedoch nützlich sein, wenn Du tiefer in die Funktionsweise des Codes eintauchen möchtest.

In [ ]:
def print_word_similarity(word_a, word_b, words, embeddings):
    """
    Gibt die Kosinusähnlichkeit zwischen den Einbettungen zweier Wörter aus.
    
    Parameter:
    wort_a (str): Das erste zu vergleichende Wort.
    wort_b (str): Das zweite zu vergleichende Wort.
    
    Gibt zurück:
    Keine
    
    Beispiel:
    print_example_word_similarity("king", "man")
    # Ausgabe: Ähnlichkeit von König und Mann: 0.61
    """
    embedding_a = [embeddings[words.index(word_a)]]
    embedding_b = [embeddings[words.index(word_b)]]
    print("Similarity of ", word_a, " and ", word_b, ":", *cosine_similarity(embedding_a, embedding_b).round(2))

In [ ]:
def get_word_suggestion(a, b, c, words, embeddings):
    """
    Finde den Wortvorschlag für eine gegebene Gleichung "a - b = x - c".

    Parameter:
    a (str): Das erste Wort in der Gleichung.
    b (str): Das zweite Wort in der Gleichung.
    c (str): Das dritte Wort in der Gleichung.

    words (list): Ein Liste mit Wörtern.
    embeddings (list): Ein Liste mit Wort-Einbettungen.

    Rückgabe:
    str: Der Wortvorschlag für die Gleichung "a - b = x - c".
    """

    lookup = dict(zip(words, embeddings))

    # Ermittelt die Worteinbettungen für a, b, und c
    vector_a = lookup.get(a)
    vector_b = lookup.get(b)
    vector_c = lookup.get(c)

    # Erstellen eines Suchraums durch Entfernen von a, b und c aus dem Lookup-Dictionary
    search_space = {
        key: value for key, value in lookup.items()
        if key not in [a, b, c]
    }

    # Berechnung der Einbettung für x durch Subtraktion von Vektor b von Vektor a und Addition von Vektor c
    embedding_x = vector_a - vector_b + vector_c

    # Erstellen Sie eine Liste aller Einbettungen im Suchraum.
    embeddings = list(search_space.values())
    embeddings = np.array(embeddings)

    # Ermittlung der Kosinusähnlichkeit zwischen Einbettung_x und allen Einbettungen im Suchraum
    similarities = cosine_similarity(embeddings, embedding_x.reshape(1, -1))

    # Ermittelt den Index der ähnlichsten Einbettung
    most_similar_index = np.argmax(similarities)

    # Ermittelt das Wort, das der ähnlichsten Einbettung entspricht.
    words = list(search_space.keys())
    most_similar_word = words[most_similar_index]

    # Ausgeben der Wortgleichung
    print(a + " - " + b + " = " + most_similar_word + " - " + c)

In [ ]:
def visualize_word_embeddings(words, embeddings):
    """
    Nimmt eine Liste von Wörtern und ihre entsprechenden Einbettungen und visualisiert sie im 3D-Raum mit PCA.
    Die Visualisierung erfolgt über ein Streudiagramm, in dem jedes Wort durch einen Punkt im 3D-Raum dargestellt wird, 
    und Wörter der gleichen Kategorie werden durch die gleiche Farbe dargestellt.

    Parameter:
    words (Liste): Eine Liste von Wörtern.
    embeddings (Liste): Eine Liste der entsprechenden Worteinbettungen.

    Rückgabe:
    Keine
    """
    word_embeddings = dict(zip(words, embeddings))

    pca = PCA(n_components=3, random_state=12345)
    pca_embeddings = pca.fit_transform(list(word_embeddings.values()))
    
    df_pca = pd.DataFrame(pca_embeddings)
    df_pca['word'] = word_embeddings.keys()
    df_pca = df_pca.rename(columns={0: 'x', 1: 'y', 2: 'z'})
    
    labels = list()
    for word in ["royal", "sex", "digits", "numbers", "car", "aircraft", "cities", "nations"]:
        for i in range(10):
            labels.append(word)
    df_pca['category'] = labels
    
    fig = px.scatter_3d(df_pca, x="x", y="y", z="z", text="word", color="category", opacity=0.9, template="plotly_dark")
    fig.update_traces(marker_size = 5)
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    if 'google.colab' in str(get_ipython()):
        fig.show(renderer='colab') 
    else:
        fig.show(renderer='iframe') 

# Text und Computer

Wenn wir Texte verarbeiten, denken wir selten darüber nach, wie wir sie in einer für Computer verständlichen Form darstellen können. Text-Repräsentation und Wort-Einbettungen sind jedoch entscheidende Konzepte in der natürlichen Sprachverarbeitung, die uns dabei helfen, genau das zu tun!

BERT (Bidirectional Encoder Representations from Transformers) ist ein fortschrittlicheres Modell zur Wort-Einbettung, das die Bedeutung von Wörtern anhand ihres Kontexts erfasst und sie in einem mehrdimensionalen Raum positioniert. Das mag sich zwar kompliziert anhören, aber es ist eigentlich ziemlich cool. Denn im Gegensatz zu One-Hot-Encodings kann BERT die Bedeutungen von Worten und deren Kontext besser darstellen. Das führt zu einer höheren Genauigkeit in der Sprachverarbeitung und hilft uns, noch besser zu verstehen, was in Texten wirklich vor sich geht.



In [ ]:
# Initialisiere BERT als Basis-Modell
transformer = TransformerWordEmbeddings('bert-base-uncased')

## Aufbau von BERT

Eine weitere faszinierende Eigenschaft von BERT ist, dass es bereits vortrainiert ist. Das bedeutet, dass es mit einer großen Menge an Texten trainiert wurde, um eine allgemeine Sprachkompetenz zu erwerben. Das ist vergleichbar mit dem Erlernen einer Sprache durch einen Menschen, der eine Vielzahl von Texten liest und somit ein besseres Verständnis für die Sprache entwickelt.

Dabei erlernt BERT ein Vokabular aus etwa 30.000 Wörtern und Subworten. Das Modell verwendet das WordPiece-Vokabular, das bedeutet, dass lange Wörter in kleinere, häufig verwendete Teile zerlegt werden, die als Subwörter bezeichnet sind. Jedes Wort oder Subwort im Vokabular wird durch einen eindeutigen Vektor dargestellt, der im Modell verwendet wird, um die Bedeutung des Wortes zu erfassen.

Ein Beispiel für ein solches Wort im WordPiece-Vokabular von BERT ist "doghouse". Da BERT mit einem unverarbeiteten Text trainiert wurde, wird das Wort in kleinere Subwörter aufgeteilt. Im Vokabular von BERT wird "doghouse" in zwei Subwörter unterteilt: "dog", und "##house" wobei das Symbol "##" anzeigt, dass das Subwort ein Teil eines längeren Wortes ist.

Jedes dieser Subwörter hat eine eindeutige ID im Vokabular und wird durch einen entsprechenden Vektor repräsentiert. Wenn BERT mit einem Text arbeitet, der das Wort "doghouse" enthält, wird das Modell jedes Subwort erkennen und deren Vektoren kombinieren, um eine umfassende Repräsentation des Wortes zu erstellen.

Zusätzlich erinnert die Architektur von BERT an den Aufbau von Neuronalen-Netzen. Es besteht aus vielen Schichten von sogenannten Encodern, die Informationen von vorherigen Schichten aufnehmen und verarbeiten, um kontextabhängige Repräsentationen von Wörtern zu erzeugen. Diese Architektur hat sich als sehr erfolgreich erwiesen und ist ein weiterer Beweis dafür, wie mächtig Neuronale-Netze in der heutigen Zeit sind.

In [ ]:
bert_vocabulary = transformer.tokenizer.vocab

print("Größe Vokabular:" , len(bert_vocabulary))
print("Größe Einbettung:", transformer.model.embeddings.word_embeddings)

print("Id für das Wort 'dog':", bert_vocabulary.get('dog'))
print("Id für das Wort '##house':", bert_vocabulary.get('##house'))
print("Id für das Wort 'house':", bert_vocabulary.get('house'))
print("Id für das Wort 'doghouse':", bert_vocabulary.get('doghouse'))

Größe Vokabular: 30522
Größe Einbettung: Embedding(30522, 768, padding_idx=0)
Id für das Wort 'dog': 3899
Id für das Wort '##house': 4580
Id für das Wort 'house': 2160
Id für das Wort 'doghouse': None


In [ ]:
# Erhalte die Schichten des Transformers
transformer_layers = transformer.model.encoder.layer
print("Transformer Schichten:", len(transformer_layers))

Transformer Schichten: 12


# King - Man = X - Woman? Ein Beispiel für Textverarbeitung

Ein klassisches Beispiel um das Konzept der Wort-Einbettungen und die Vorteile der BERT-Einbettungen zu demonstrieren ist das Rätsel: "king - man = x - woman". Dabei werden die Vektoren der Wörter "king", "man", "woman" und "queen" sinnbildlich in Bezug zueinander gestellt, um das Wort "x" zu erhalten. Die Intuition dahinter ist, dass die Differenz zwischen "king" und "man" die Eigenschaften repräsentiert, die einzigartig für das Wort "king" sind, und das Hinzufügen der Eigenschaften von "woman" zu dieser Differenz die Eigenschaften ergibt, die einzigartig für das Wort "queen" sind.

Der Vorteil in der Verwendung von BERT-Einbettungen besteht darin, dass sie auf einem starken Sprachmodell basieren, und so in der Lage sind, anspruchsvollere und nuanciertere Beziehungen (Semantik) zwischen Wörtern erfassen.

In [ ]:
# Hier siehst du einen Text aus jeweils 10 Worten für 8 Klassen, welche die 4 Kategorien Menschen, Zahlen, Fahrzeuge und Länder beschreiben.
royal = "royal monarchy crown queen king coronation throne palace majestic imperial"
sex = "sex man woman diversity transgender masculinity femininity intersex LGBTQ gender"

digits = "1 2 3 4 5 6 7 8 9 10"
numbers = "one two three four five six seven eight nine ten"

car = "car wheels engine speed road drive auto fuel vehicle emission"
aircraft = "aircraft airbus boeing B747 A380 pilot cockpit wings takeoff landing"

cities = "Beijing Tokyo London Paris Berlin Rome Madrid Warsaw Bangkok Bern"
nations = "China Japan UK France Germany Italy Spain Poland Thailand Switzerland"

corpus = " ".join([royal, sex, digits, numbers, car, aircraft, cities, nations])

print("Text of all words:", corpus)

Text of all words: royal monarchy crown queen king coronation throne palace majestic imperial sex man woman diversity transgender masculinity femininity intersex LGBTQ gender 1 2 3 4 5 6 7 8 9 10 one two three four five six seven eight nine ten car wheels engine speed road drive auto fuel vehicle emission aircraft airbus boeing B747 A380 pilot cockpit wings takeoff landing Beijing Tokyo London Paris Berlin Rome Madrid Warsaw Bangkok Bern China Japan UK France Germany Italy Spain Poland Thailand Switzerland


## Vom Wort zum Vektor

Das Sentence-Objekt in Flair hilft dabei, Texte und Sätze als Vektoren darzustellen. Das ist hilfreich, um Texte zu analysieren und verschiedene Aufgaben in der Sprachverarbeitung zu lösen.

Um Vektoren mit dem Sentence-Objekt zu erstellen, wird einfach die .embed() Methode aufgerufen. Dabei werden vorab trainierter BERT-Embedding-Vektoren verwendet, die dann für die jeweiligen Worte zusammengesetzt werden. Das ist ähnlich wie der Aufruf von fit_transform() in der sklearn Bibliothek, wo ein vorab trainiertes Modell die gegebenen Daten in eine andere Darstellung transformiert.

In [ ]:
# Hier wird aus unserem Korpus ein Satz aus einzelnen Worten gebildet.
sentence = Sentence(corpus)

# Hier wird für jedes einzelne Wort eine Einbettung erzeugen.
transformer.embed(sentence)

# Hier werden für die Wörter und Embeddings separate Listen erstellt.
words = list()
embeddings = list()

for token in sentence:
    word = token.text
    embedding = token.embedding.numpy()
    words.append(word)
    embeddings.append(embedding)

print("Beispiel Wort:", words[0])
print("Einbettung Wort:", embeddings[0][:4])

Beispiel Wort: royal
Einbettung Wort: [ 0.8874518   0.17805542  0.58898306 -0.08429976]


## Semantische Ähnlichkeit von Wörtern

Semantische Ähnlichkeit ist eine Methode zur Messung der Ähnlichkeit zwischen Bedeutungen von Wörtern. Es hilft uns, in Anwendungen der Textverarbeitung besser zu machen, indem es uns ermöglicht, die Bedeutung von Wörtern zu verstehen und darzustellen.

Daher kännen wir dieses Konzept verwenden, um die Gleichung "king - man = c - woman" mit BERT-Einbettungen zu lösen, indem die Ähnlichkeiten zwischen den Wörtern in der Gleichung verglichen werden.

In diesem Gleichnis geht es darum, das Wort "x" zu finden, das dem Wort "king" am ähnlichsten ist, während es dem Wort "man" am unähnlichsten ist. In ähnlicher Weise sollte das Wort "woman" dem Wort "man" am ähnlichsten sein.

Um das Wort "x" zu finden, können wir schlichtweg die Kosinus-Ähnlichkeit zwischen den BERT-Einbettungen der einzelnen Wortpaare in der Gleichung berechnen.
Diese ist für eine solche Aufgabe gut geeignet, da sie den Kosinus des Winkels zwischen zwei Vektoren in einem hochdimensionalen Raum misst, wobei jeder Vektor die Einbettung eines Wortes darstellt.

Die Kosinus-Ähnlichkeitsmetrik reicht von 0 bis 1, wobei 1 bedeutet, dass zwei Wörter genau dieselbe Bedeutung haben, und 0 bedeutet, dass die beiden Wörter völlig unterschiedliche Bedeutungen haben.

In [ ]:
# Hier siehst Du, dass Worte ähnlich oder unähnlich sein können.
print_word_similarity("king", "queen", words, embeddings)
print_word_similarity("man", "queen", words, embeddings)
print_word_similarity("woman", "queen", words, embeddings)

Similarity of  king  and  queen : [0.88]
Similarity of  man  and  queen : [0.53]
Similarity of  woman  and  king : [0.5]
Similarity of  woman  and  queen : [0.54]


In [ ]:
# Hier siehst du, dass die Wort-Einbettungen einer gewissen Logik folgen.
get_word_suggestion("king", "man", "woman", words, embeddings)
get_word_suggestion("3", "1", "2", words, embeddings)
get_word_suggestion("three", "one", "two", words, embeddings)
get_word_suggestion("aircraft", "wings", "wheels", words, embeddings)
get_word_suggestion("Germany", "Berlin", "Bangkok", words, embeddings)

king - man = queen - woman
3 - 1 = 4 - 2
three - one = four - two
aircraft - wings = vehicle - wheels
Germany - Berlin = Thailand - Bangkok


In [ ]:
# Hier kannst du einen interaktiven Plot der Wöter und deren Bedeutung im 3-Dimensionalen betrachten.
visualize_word_embeddings(words, embeddings)

# Hausaufgaben

Frage 1:
Was ist ein Wort-Einbettungsmodell?

    A) Eine Methode zur Messung der Ähnlichkeit zwischen Wörtern
    B) Eine Methode zur Visualisierung der Verteilung von Wörtern in einem Vektorraum
    C) Eine Methode zur Repräsentation der Bedeutung von Wörtern als Vektoren
    D) Eine Methode zur Erstellung von Textkorpora aus Rohdaten
    Antwort: C

Frage 2:
Welche Methode wird verwendet, um die Bedeutung von Wörtern in einem BERT-Modell zu lernen?

    A) TF-IDF
    B) Bag-of-Words-Modell
    C) Word2Vec
    D) Transformer-Netzwerke
    Antwort: D

Frage 3:
Was ist der Unterschied zwischen BERT und Word2Vec?

    A) BERT verwendet neuronale Netze, während Word2Vec ein statistisches Modell ist.
    B) BERT kann bidirektionale Kontexte verwenden, während Word2Vec nur unidirektionale Kontexte verwenden kann.
    C) BERT kann nur die Bedeutung von Wörtern, nicht aber deren Beziehungen, erfassen, während Word2Vec beides kann.
    D) BERT verwendet lineare Algebra, während Word2Vec probabilistische Modelle verwendet.
    Antwort: B